In [1]:
import os,git,shutil
import pandas as pd

In [2]:
# 指定されたデータサイズでファイルを分割する
def divide_file(filePath, chunkSize):
    readedDataSize = 0
    i = 0
    fileList = []
    # 対象ファイルを開く
    f = open(filePath, "rb")
    # ファイルを読み終わるまで繰り返す
    contentLength = os.path.getsize(filePath)
    while readedDataSize < contentLength:
        # 読み取り位置をシーク
        f.seek(readedDataSize)
        # 指定されたデータサイズだけ読み込む
        data = f.read(chunkSize)
        # 分割ファイルを保存
        saveFilePath = filePath + "." + str(i)
        with open(saveFilePath, 'wb') as saveFile:
            saveFile.write(data)
        # 読み込んだデータサイズの更新
        readedDataSize = readedDataSize + len(data)
        i = i + 1
        fileList.append(saveFilePath)
    return fileList

# 渡されたファイルリストの順序で１つのファイルに結合する
def join_file(fileList, filePath):
    with open(filePath, 'wb') as saveFile:
        for f in fileList:
            data = open(f, "rb").read()
            saveFile.write(data)
            saveFile.flush()

In [55]:
class env_util():
    def __init__(self):
        self.u_path = os.environ["userprofile"]
        self.miniconda_path = os.path.join(self.u_path,"miniconda3","pkgs")
        self.fac_path = os.path.join(self.u_path,"000_work","env_factory_miniconda39")
        self.fac_pkg_path = os.path.join(self.fac_path,"pkgs")
        self.repo_fac = git.Repo(self.fac_path)
        self.remote_folder_path = os.path.join(self.u_path,"000_work","env_miniconda_39")
        self.repo_remote_folder = git.Repo(self.remote_folder_path)

    def comfirm_file(self):
        a = self.repo_fac.git.status()
        b = a.split(" to include in what will be committed)\n\t")[1]
        b = b.split("\n\nno changes added to commit")[0]
        c = b.split("\n\t")
        print(f"未コミットファイルは{len(c)}個です")
        return c

    def get_list(self):
        file_list = []
        file_list_batsu = []
        for root, dirs, files in os.walk(self.miniconda_path):
            for filename in files:
                fname = os.path.join(root, filename)
                file_list.append([fname,os.path.getsize(fname) / 1000000])
            df = pd.DataFrame(file_list)
            df.columns=["path","MB"]
        return df
        
if __name__ == '__main__':
    self = env_util()

# push出来る状態にする

## 初期化
factoryの中に未コミットがないことを確認

In [56]:
li = self.comfirm_file()

未コミットファイルは172個です


未コミットがあれば、コミットとして0にする

In [ ]:
if len(li) !=0:
    pass


## miniconda環境を上書きコピーする

In [22]:
files = os.listdir(self.miniconda_path)
file_list = [f for f in files if os.path.isfile(os.path.join(self.miniconda_path, f))]
for file in file_list:
    f_path_moto = os.path.join(self.miniconda_path,file)
    f_path_copy = os.path.join(self.fac_pkg_path,file)
    try:
        shutil.copyfile(f_path_moto,f_path_copy)
    except :
        print(f"エラー:{file}")
        pass

In [23]:
li = self.comfirm_file()

未コミットファイルは172個です


## envにコピー

In [31]:
file_list = li
out_list = []
for file in file_list:
    f_path_moto = os.path.join(self.fac_path,file)
    f_path_copy = os.path.join(self.remote_folder_path,file)
    out_list.append([f_path_copy,os.path.getsize(f_path_moto) / 1000000])
    try:
        shutil.copyfile(f_path_moto,f_path_copy)
    except :
        print(f"エラー:{f_path_moto}")
        pass
df = pd.DataFrame(out_list)
df.columns=["path","MB"]

In [34]:
df_b = df[df["MB"]>99]
df_b

,path,MB
99,C:\Users\M151325\000_work\env_miniconda_39\pkg...,180.784978
143,C:\Users\M151325\000_work\env_miniconda_39\pkg...,111.249412


## 100M以上は分割

In [ ]:
if len(df_b)!=0:
    for target in files = list(df_b["path"]):
        fileList = divide_file(target, 50*10**6)
        df_a = pd.DataFrame(fileList,columns=["path"])
        df_a.to_csv(os.path.join("sep_file_list/",os.path.basename(target)+".csv"),index=False)
        os.remove(target)

# gitへpush

In [59]:
self.repo_remote_folder.git.status()

'On branch main\nYour branch is up to date with \'origin/main\'.\n\nUntracked files:\n  (use "git add <file>..." to include in what will be committed)\n\t.ipynb_checkpoints/\n\tenv.md\n\tpkgs/\n\tsep_file_list/\n\tstart_env/\n\tutility.ipynb\n\nnothing added to commit but untracked files present (use "git add" to track)'

# 100M以上を復活

In [48]:
join_file(fileList, target+"_join")

# インストール

# 未コミットを確認

In [21]:
self.comfirm_file()

未コミットファイルは172個です
